In [1]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/nfl_offense_cleaned']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

This is a python tutorial designed for the use of pandas in a data frame. The data comes from the open data set /kaggle/input/nfl-offense-cleaned-2017to2007/nfl_offense_cleaned.csv . 

The purpose of this tutorial is to practice python/pandas while also trying to figure out which player is the best passer between the years 2007 and 2017. The tutorial will lead particpants through the different passer catagories such as (PCT: completion percentage, COMP: Total Completions, RATE:Passer rating).

Hopefully at the end of this tutorial users will feel more confident with pandas and know a little bit more about players during this time frame.

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
exec(os.environ['IREWR_IMPORTS'])
# FIRST-AUTHOR: remove plotting
# import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# FIRST-AUTHOR: remove path printing
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:
NFL = pd.read_csv('./input/nfl_offense_cleaned.scaled.csv')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/26 22:12:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/26 22:12:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/01/26 22:12:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/01/26 22:12:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/01/26 22:12:44 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


*Hint* Run the the top two code cells to avoid getting errors in later problems. 

**1. Summerize data set**

**1.1** What are the first 5 values in the NFL data frame?

In [4]:
NFL.head()

,RK,PLAYER,POS,TEAM,COMP,ATT,PCT,YDS,YDS/A,LONG,TD,INT,SACK,RATE,YDS/G,YEAR
0,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Tom Brady,QB,NE,279.0,408.0,68.4,3374.0,8.27,64.0,26.0,3.0,24.0,111.7,307.0,2017.0
2,2,Kirk Cousins,QB,WSH,249.0,376.0,66.2,3038.0,8.08,74.0,19.0,6.0,31.0,101.1,276.0,2017.0
3,3,Drew Brees,QB,NO,266.0,373.0,71.3,3029.0,8.12,54.0,16.0,5.0,13.0,104.1,275.0,2017.0
4,None,Russell Wilson,QB,SEA,256.0,411.0,62.3,3029.0,7.37,66.0,23.0,8.0,26.0,95.2,275.0,2017.0


*Will return the first five rows of the dataset to make sure data is being presented correctly*

**1.2** How many rows and columns are there?

In [5]:
NFL.shape

(2270, 16)

*There are 1081 rows and 16 columns in the data set*

## 2. Basic Analysis

**2.1** Return the five players who have thrown for the most yards.

In [6]:
evaluate_lazy(NFL.sort_values('YDS', ascending=False).iloc[:5])

/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,RK,PLAYER,POS,TEAM,COMP,ATT,PCT,YDS,YDS/A,LONG,TD,INT,SACK,RATE,YDS/G,YEAR
377,1,Peyton Manning,QB,DEN,450.0,659.0,68.3,5477.0,8.31,78.0,55.0,10.0,18.0,115.1,342.0,2013.0
1458,1,Peyton Manning,QB,DEN,450.0,659.0,68.3,5477.0,8.31,78.0,55.0,10.0,18.0,115.1,342.0,2013.0
555,1,Drew Brees,QB,NO,468.0,657.0,71.2,5476.0,8.34,79.0,46.0,14.0,24.0,110.6,342.0,2011.0
1636,1,Drew Brees,QB,NO,468.0,657.0,71.2,5476.0,8.34,79.0,46.0,14.0,24.0,110.6,342.0,2011.0
556,2,Tom Brady,QB,NE,401.0,611.0,65.6,5235.0,8.57,99.0,39.0,12.0,32.0,105.6,327.0,2011.0


*Sorted from the highest yards to the lowest yards in the top 5. 

*Total yards is one of the best ways to see how good a player was over a season*

**2.2** Which player has the most completions in a season?

In [7]:
NFL.loc[NFL.COMP.idxmax()].PLAYER

'Drew Brees'

*Based on the inputs returns only the player's name that has the most completions*

*We can assume that total completions for a season is a good measurement for how good a player is, but should we also consider completion percent(PCT) or attempts?*

**2.3**Sort the longest passes from shortest to longest.

In [8]:
evaluate_lazy(NFL.groupby("LONG").max())

/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,RK,PLAYER,POS,TEAM,COMP,ATT,PCT,YDS,YDS/A,TD,INT,SACK,RATE,YDS/G,YEAR
LONG,,,,,,,,,,,,,,,
-8.0,87,Shann Schillinger,DB,ATL/TEN,1.0,1.0,100.0,-8.0,-8,0.0,0.0,0.0,79.2,-2.0,2013.0
-6.0,103,Anquan Boldin,WR,BAL,1.0,1.0,100.0,-6.0,-6,0.0,0.0,0.0,79.2,0.0,2010.0
-4.0,78,Jon Ryan,P,SEA,1.0,1.0,100.0,-4.0,-4,0.0,0.0,0.0,79.2,0.0,2017.0
-3.0,89,Sam Koch,P,BAL,1.0,1.0,100.0,-3.0,-3,0.0,0.0,0.0,79.2,0.0,2015.0
-2.0,103,Kevin Faulk,RB,NE,1.0,1.0,100.0,-2.0,-2,0.0,0.0,0.0,79.2,0.0,2008.0
0.0,92,Willie Parker,WR,WSH,1.0,5.0,100.0,0.0,0,0.0,1.0,1.0,79.2,0.0,2017.0
1.0,83,Ray Rice,RB,BAL,1.0,1.0,100.0,1.0,1,1.0,0.0,0.0,118.8,0.0,2011.0
2.0,91,Tyrod Taylor,WR,TB,1.0,6.0,100.0,2.0,2,1.0,1.0,2.0,118.8,2.0,2017.0
3.0,88,Matt Gutierrez,WR,TB,2.0,5.0,100.0,6.0,3,1.0,0.0,0.0,118.8,3.0,2015.0


*organizes the players who have value for LONG and is sorted from shortest to longest pass*

*Do we see a correlation between longest passses and any other catagories, does this make for a good way to judge players?*

**2.4**What are the names of players who have a perfect passer rating (158.3)?

In [9]:
evaluate_lazy(NFL.query('RATE > 158.2').loc[:, ['PLAYER']])

/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,PLAYER
52,Mohamed Sanu
55,Pat O'Donnell
60,Tarik Cohen
144,Willie Snead
152,Marqise Lee
159,Doug Baldwin
246,Ryan Nassib
249,Antonio Andrews
254,Cecil Shorts III
334,Mohamed Sanu


*Returns only the names of players who have a perfect passer rating on the season*

**2.5** What are the average values for all columns in the data frame?

In [10]:
evaluate_lazy(NFL.mean().round(2))

COMP      112.66
ATT       182.66
PCT        56.19
YDS      1295.73
LONG       41.95
TD          7.90
INT         4.90
SACK       11.99
RATE       77.89
YDS/G     120.98
YEAR     2012.03
dtype: float64

*notice that no string columns are present, because pandas is smart enough to only conduct mean on integers and floats*

*we can judge players and see if their individual values are greater or less than the average*

** 3. Summerize**

**3.1**Plot a histogram of number of players based on their attempts.

In [11]:
# FIRST-AUTHOR: remove plotting
# NFL.ATT.plot.hist()
NFL.ATT

0        NaN
1      408.0
2      376.0
3      373.0
4      411.0
5      395.0
6      361.0
7      388.0
8      396.0
9      361.0
10     369.0
11     354.0
12     349.0
13     330.0
14     343.0
15     395.0
16     350.0
17     324.0
18     323.0
19     347.0
20     306.0
21     348.0
22     308.0
23     267.0
24     259.0
25     313.0
26     268.0
27     319.0
28     204.0
29     271.0
30     224.0
31     193.0
32     205.0
33     184.0
34     178.0
35     163.0
36     127.0
37     140.0
38     125.0
39     117.0
40      91.0
41       NaN
42      75.0
43      72.0
44      43.0
45      43.0
46      42.0
47      24.0
48      18.0
49      23.0
50      16.0
51       4.0
52       1.0
53       1.0
54      14.0
55       1.0
56       3.0
57       2.0
58       5.0
59       1.0
60       1.0
61       2.0
62       7.0
63       1.0
64       1.0
65       3.0
66       2.0
67       NaN
68       1.0
69       1.0
70       1.0
71       1.0
72       2.0
73       1.0
74       1.0
75       1.0
76       1.0

*The data has every pass thrown during the decade, most of the players dont have more than 10 passes which is why there is such a large spike in the first bin*

**3.2**Create a pie chart that breaks down the count of each position in the data frame. 

In [12]:
# FIRST-AUTHOR: remove plotting
# NFL.POS.value_counts().plot.pie()
NFL.POS.value_counts()

/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(


 QB    1667
 WR     243
 RB     134
 P      109
 PK      14
 TE      10
 S        9
 FB       4
 NT       2
 CB       2
 DB       2
 LB       2
Name: POS, dtype: int64

*Notice that most of the players have the position of a QB(quarterback)*

**So who do you think is the best player during this time frame?**